In [30]:
import numpy as np
from itertools import combinations as combowombo

In [2]:
with open('data/d_quite_big.in') as file:
    data = file.read().split()

data = [int(_) for _ in data]
MAXSLICES = data[0]
TYPES = data[1]
PIZZAS = data[2:]
pizza_indices = {pizza: index for index, pizza in enumerate(PIZZAS)}

In [96]:
def create_prob_list(size = 1, dist = np.random.uniform, **kwargs):
    prob_list = sorted(dist(size = size, **kwargs))
    total = sum(prob_list)
    prob_list = prob_list/total
    return prob_list

In [28]:
def all_slices(pizza_list):
    possible_slices = {sum(comb): list(comb) for k in range(1,len(pizza_list)+1) for comb in combowombo(pizza_list,k)}
    return possible_slices

In [97]:
def compute_slices(max = MAXSLICES, pizza_list = PIZZAS):
    final_list = []
    while (max >= 0):
      admissible_pizzas = [pizza for pizza in pizza_list if pizza <= max]
      if admissible_pizzas:
        num_pizzas = len(admissible_pizzas)
        prob_list = create_prob_list(size=num_pizzas, dist=np.random.exponential, scale = 0.0001)
        index = np.random.choice(num_pizzas,p=prob_list)
        rand_pizza = admissible_pizzas.pop(index)
        pizza_list = admissible_pizzas
        max = max - rand_pizza
        final_list.append(rand_pizza)
      else:
        break
    return final_list

In [98]:
MAX = 1000000000
best_output = []
best_total = 0
for i in range(100):
    output = compute_slices(max = MAX)
    total = sum(output)
    if total > best_total:
        best_total = total
        best_output = output
print(f'Total = {best_total} \ndiff = {MAX - best_total}')